In [ ]:
# Lake User Demo Notebook 
# List of activities 
# 1 - Connect to cms_secured_db glue database 
# 2 - Query specific tables and columns to verify the lakeformation tag based access control
# 3 - security_tag:sec-5 - Should allow to read from the database and tables
# 4 - security_tag:sec-4 - Should restrict specific tables from accessing
# 5 - security_tag:sec-2 - Should restrict specific table columns from accessing


In [ ]:
%reload_ext sql


In [ ]:
# Imports
import json 
import boto3
import logging
import pprint
from pandas import DataFrame
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()

In [ ]:
# import orbit helpers
from aws_orbit_sdk.database import get_athena
from aws_orbit_sdk.common import get_workspace,get_scratch_database
workspace = get_workspace()

team_space = workspace['team_space']
assert team_space == 'lake-user'

catalog_id = workspace['EksPodRoleArn'].split(':')[-2] 

In [ ]:
lfc = boto3.client('lakeformation')
iamc = boto3.client('iam')
ssmc = boto3.client('ssm')
gluec = boto3.client('glue')

In [ ]:
secured_glue_db = "cms_secured_db"

In [ ]:
%reload_ext sql
%config SqlMagic.autocommit=False # for engines that do not support autommit
athena = get_athena()
%connect_to_athena -database default

In [ ]:
%sql show databases

In [ ]:
%connect_to_athena -database cms_secured_db


In [ ]:
%catalog -database cms_secured_db

# Establish Athena Connection

In [ ]:
%reload_ext sql
athena_url = athena.get_connection_to_athena('secured_glue_db')['db_url']
athena_url

In [ ]:
%sql $athena_url

In [ ]:
%%sql $athena_url

SELECT 1 as "Test"

# Demonstrate use of Secured LakeFormation Databases

## Step-1- lets read from a secured database table with sec-5 tag

In [ ]:
%%sql $athena_url secured_carrier_claims <<

select * from cms_secured_db.carrier_claims limit 2

In [ ]:
secured_carrier_claims

## Step-2- lets read from secured table with sec-4

In [ ]:
cms_secured_db_response=%catalog -database cms_secured_db

In [ ]:
cms_secured_db_tables_for_lake_user = [table_name for table_name in cms_secured_db_response.data.keys() ]

In [ ]:
assert ('inpatient_claims' not in cms_secured_db_tables_for_lake_user)

In [ ]:
%%sql $athena_url secured_inpatient_claims <<
select * from cms_secured_db.inpatient_claims limit 1

In [ ]:

try: secured_inpatient_claims
except NameError: 
    print('Success')

## Step-3- lets read from secured columns with sec-2

In [ ]:
%%sql $athena_url secured_beneficiary_summary <<
select * from cms_secured_db.beneficiary_summary limit 1

In [ ]:
assert('sp_depressn' not in secured_beneficiary_summary.field_names)
assert('sp_diabetes' not in secured_beneficiary_summary.field_names)

In [ ]:
secured_beneficiary_summary

## Check secured data  bucket access

In [ ]:
!aws s3 ls s3://orbit-dev-env-secured-demo-lake-044923722733-smqduj/

# End of Lake User demo notebook